In [1]:
dataset_path = '/mnt/edisk/dataset/unimodel_calibrate'
model_path = 'unimodel.optimized.onnx'
qmodel_path = 'unimodel.quantized.onnx'

In [2]:
import os
import pickle

from tqdm import tqdm

import numpy as np
import torch
from torch.utils.data import DataLoader

from torch.ao.quantization.observer import HistogramObserver, PerChannelMinMaxObserver
from torch.ao.quantization.fake_quantize import FakeQuantizeBase, FakeQuantize

from onnxutils.quantization import symbolic_trace, ModuleQuantizer, compute_metric, mse_kernel, cosine_kernel, snr_kernel

from onnxutils.common import DatasetUtils
from onnxutils.onnx import OnnxModel
from onnxutils.onnx2torch import convert
from onnxutils.onnx2torch.scatter_nd import TorchScatterNd

from unimodel_pipeline import UnimodelDataset

# Prepare

In [3]:
onnx_model = OnnxModel.from_file(model_path)
with onnx_model.session() as sess:
    for node in onnx_model.proto().graph.node:
        if node.name == '':
            node.name = sess.unique_name()
onnx_model.topological_sort()
torch_model = convert(onnx_model)

dataset = UnimodelDataset(dataset_path, torch_model.onnx_mapping.inputs)

# Quantization

In [4]:
graph_model = pickle.loads(pickle.dumps(torch_model))
graph_model = symbolic_trace(graph_model, skipped_module_classes=[TorchScatterNd])

quantizer = ModuleQuantizer()
graph_model = quantizer.quantize(graph_model, [
    {
        'name': 'imgs',
        'activation': FakeQuantize.with_args(observer=HistogramObserver),
    },
    {
        'module_name': 'backbone/stage1/conv/Conv',
        'weight': FakeQuantize.with_args(observer=PerChannelMinMaxObserver),
    },
    {
        'module_name': 'backbone/stage1/act/Relu',
        'activation': FakeQuantize.with_args(observer=HistogramObserver),
    },
])

graph_model.print_readable()

/home/ycgao/Workdir/onnxutils/onnxutils/onnx2torch/resize.py:45: UserWarning: `nn.functional.upsample_bilinear` is deprecated. Use `nn.functional.interpolate` instead.
  x = nn.functional.upsample_bilinear(x, self.sizes)


class GraphModule(torch.nn.Module):
    def forward(self, distortion_coeff, extrinsics, fused_projection, imgs, mpp, mpp_pose_state, mpp_valid, norm_intrinsics, pose, prev_bev_feats, prev_feat_stride16, prev_pose_inv, sdmap_encode, sdmap_mat):
        # No stacktrace found for following nodes
        fq0 = self.fq0(imgs);  imgs = None
        
         # File: /opt/miniconda3/lib/python3.10/site-packages/torch/fx/proxy.py:219 in create_proxy, code: proxy.node.stack_trace = ''.join(CapturedTraceback.extract().format())
        backbone_stage1_conv_conv = getattr(self, "backbone/stage1/conv/Conv")(fq0);  fq0 = None
        
         # File: /opt/miniconda3/lib/python3.10/site-packages/torch/fx/proxy.py:219 in create_proxy, code: proxy.node.stack_trace = ''.join(CapturedTraceback.extract().format())
        random_861d8f24_d0f0_11ef_aeac_8586c47ece25_5 = self.random_861d8f24_d0f0_11ef_aeac_8586c47ece25_5(backbone_stage1_conv_conv);  backbone_stage1_conv_conv = None
        
         # Fil

'class GraphModule(torch.nn.Module):\n    def forward(self, distortion_coeff, extrinsics, fused_projection, imgs, mpp, mpp_pose_state, mpp_valid, norm_intrinsics, pose, prev_bev_feats, prev_feat_stride16, prev_pose_inv, sdmap_encode, sdmap_mat):\n        # No stacktrace found for following nodes\n        fq0 = self.fq0(imgs);  imgs = None\n        \n         # File: /opt/miniconda3/lib/python3.10/site-packages/torch/fx/proxy.py:219 in create_proxy, code: proxy.node.stack_trace = \'\'.join(CapturedTraceback.extract().format())\n        backbone_stage1_conv_conv = getattr(self, "backbone/stage1/conv/Conv")(fq0);  fq0 = None\n        \n         # File: /opt/miniconda3/lib/python3.10/site-packages/torch/fx/proxy.py:219 in create_proxy, code: proxy.node.stack_trace = \'\'.join(CapturedTraceback.extract().format())\n        random_861d8f24_d0f0_11ef_aeac_8586c47ece25_5 = self.random_861d8f24_d0f0_11ef_aeac_8586c47ece25_5(backbone_stage1_conv_conv);  backbone_stage1_conv_conv = None\n        

In [5]:
# calibration
for m in graph_model.modules():
    if isinstance(m, FakeQuantizeBase):
        m.disable_fake_quant()

dataloader = DataLoader(
    DatasetUtils.take_front(
        DatasetUtils.transform(
            dataset,
            lambda items: tuple(x.to('cuda') for x in items)
        ),
        256
    ),
    batch_size=None
)
graph_model.eval().to('cuda')
for data in tqdm(dataloader):
    graph_model(*data)

for m in graph_model.modules():
    if isinstance(m, FakeQuantizeBase):
        m.enable_fake_quant()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [00:24<00:00, 10.63it/s]


# Analysis

In [6]:
dataloader = DataLoader(
    DatasetUtils.take_front(
        DatasetUtils.transform(
            dataset,
            lambda items: tuple(x.to('cuda') for x in items)
        ),
        256
    ),
    batch_size=None
)
torch_model.eval().to('cuda')
graph_model.eval().to('cuda')

for data in dataloader:
    gt = torch_model(*data)
    pred = graph_model(*data)

    for name, metric, val0, val1 in zip(torch_model.onnx_mapping.outputs, compute_metric(gt, pred, cosine_kernel), gt, pred):
        print(name, metric)
    break

/home/ycgao/Workdir/onnxutils/onnxutils/onnx2torch/resize.py:45: UserWarning: `nn.functional.upsample_bilinear` is deprecated. Use `nn.functional.interpolate` instead.
  x = nn.functional.upsample_bilinear(x, self.sizes)


allsign_flatten_bboxes [0.9999963045120239, 0.9999945163726807, 0.9999936819076538]
allsign_flatten_cls_scores [0.9955169558525085, 0.9944993853569031, 0.9902694225311279]
allsign_flatten_objectness [0.99916672706604, 0.9986743927001953, 0.9984148144721985]
bev2d_cls_result [0.9998071193695068]
bev2d_semantic_cls_result [0.9942261576652527]
det3d_bbox_score [0.9997676610946655]
det3d_bboxes_attr_scores [0.9999969005584717]
det3d_bboxes_category [0.9790540933609009]
det3d_bboxes_subcategory [0.9997053146362305]
det3d_decoded_bboxes [0.99994957447052]
ego_velocity [1.0]
feats_0 [0.999282717704773, 0.9992157220840454, 0.9992157220840454, 0.9992514848709106, 0.9991780519485474, 0.9992655515670776, 0.9992352724075317, 0.9991344809532166, 0.999208927154541, 0.9990980625152588]
feats_1 [0.9990477561950684, 0.9988771677017212, 0.9987227916717529, 0.9990764856338501, 0.9987497329711914, 0.9989275932312012, 0.9989163875579834, 0.9970071315765381, 0.9989393949508667, 0.9984675645828247]
feats_2 [